# lab-10-5-1 Advanced-CNN(VGG)
<br></br>
### VGG-net

Oxford VGG 에서 만든 이미지 분류 모델

3x3 kernel, stride=1, padding=1 인 CNN 레이어로만 구성된 비교적 간단한 형태의 모델로 네트워크의 깊이 차이에 따른 학습 성능을 비교할 수 있도록 제작되었다

In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [2]:
__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]

# trained models
model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

In [3]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features  # Convolution
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        # 원하는 클래스 개수를 출력값으로 가지는 Fully Connected Layer
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        
        # 가중치 초기화
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) # 여러 층으로 구성된 Convolution layers 
        x = self.avgpool(x)  # avg pool
        x = x.view(x.size(0), -1)
        x = self.classifier(x) # Fully Connected layer
        return x

    # 가중치 초기화
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):  # m == conv2d 층인가?
                # kaiming he가 제안한 초기화 (정규분포)
                # std(표준편차) = gain / root(fan_mode)
                # fan in은 forward시에 weight 변동을 방지
                # fan out은 backward시에 weight 변동을 방지
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0) # 지정한 value로 bias 초기화
            elif isinstance(m, nn.BatchNorm2d):  # m == batchnorm 층인가?
                nn.init.constant_(m.weight, 1)  # 지정한 value로 weight 초기화
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):  # m == linear 층인가?
                nn.init.normal_(m.weight, 0, 0.01)  # 정규분포로 초기화 (평균, 표준편차)
                nn.init.constant_(m.bias, 0)

In [4]:
# 'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3  # RGB
    
    # cfg dict에 입력된 순서대로 Convolution층과 MaxPool, BatchNorm층을 쌓음
    # 활성화 함수는 ReLU 사용
    for v in cfg:
        if v == 'M':
            # MaxPool
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            # Convolution
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                # batch_norm == True
                # 배치 정규화와 ReLU 활성화 함수 층을 쌓음
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                # batch_norm == False
                # ReLU 활성화 함수 층만 쌓음
                layers += [conv2d, nn.ReLU(inplace=True)]
            # 현재 v값은 다음 층의 input channel의 값이 되도록 해줌
            in_channels = v
                     
    return nn.Sequential(*layers)

In [5]:
# dict에 VGG넷 모델의 구조를 여러개 적어놓고 각각 적용할 수 있게 함
# Num : out channel size, M : Max Pooling
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], #8 + 3 =11 == vgg11
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'], # 10 + 3 = vgg 13
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'], #13 + 3 = vgg 16
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'], # 16 +3 =vgg 19
    'custom' : [64,64,64,'M',128,128,128,'M',256,256,256,'M']
}

In [6]:
# cfg에 입력한 형태대로 conv, maxpool 층 제작, 배치 정규화는 True로 설정
conv = make_layers(cfg['custom'], batch_norm=True)

In [9]:
# 제작된 conv를 VGG에 feature로 입력하고 분류 클래스는 10개, 가중치 초기화는 True로 설정
CNN = VGG(conv, num_classes=10, init_weights=True)

In [10]:
# 완성된 모델의 형태 출력
CNN

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(128, eps=1e-0